In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 12 12:04:31 2017

@author: gama
"""
from  keras.utils import multi_gpu_model
import tensorflow as tf
import re
import numpy as np
from keras.models import Model
from keras.layers import Embedding,Masking
from keras.layers import Input, Dense,Reshape,concatenate,Flatten,Activation,Permute,multiply
from keras.layers import GRU,Conv1D,GlobalMaxPooling1D,TimeDistributed,RepeatVector,LSTM,MaxPooling1D
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.layers import Lambda,Dropout
from keras.utils import to_categorical,multi_gpu_model
import gc
import random
import nltk
import math
from tqdm import tqdm,tqdm_notebook
import csv
import keras.backend as K
from keras.models import load_model
import math

In [11]:
#setting tf_config
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=True))
K.set_session(sess)
# jieba.load_userdict('dict.txt.big.txt')
# jieba.load_userdict('NameDict_Ch_v2.txt')


In [30]:
post=[]
response=[]
post_file=open('QA dataset/stc_weibo_train_post','r')
for i in post_file.readlines():
    post.append(i.split())

post_file=open('QA dataset/stc_weibo_train_response','r')
for i in post_file.readlines():
    response.append(i.split())


In [31]:
print(len(post),len(response))

4435959 4435959


In [32]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
token_stream = []
que_pad=20
ans_pad=10
stop_count=0
pair_train1=[]
pair_train2=[]
check_stop=[]
count=0

for key,i in enumerate(post):
    if len(pair_train1)>=100000:
        break
    if len(i)>=ans_pad or len(response[key])>=ans_pad or len(i)<3 or len(response[key])<3:
        continue
    while len(i)<que_pad:
        i.append('PAD')
    while len(response[key])<ans_pad:
        response[key].append('PAD')    
    token_stream.extend(i)
    pair_train1.append(i)    
    token_stream.extend(response[key])
    pair_train2.append(response[key])    


In [33]:
print('num_of_pairs',len(pair_train1))
pair=len(pair_train1)
#TOP=['PAD','EOS']             
#TOP.extend(token_stream)
words=list(set(token_stream))
words.remove('PAD')
#del token_stream

word2idx={}
word2idx['PAD']=0
for i, word in enumerate(words):
    word2idx[word]=i+1
num_words = len(word2idx)
print("num_words:")
print(num_words)
                    
print('process_data')

predict_pair=ans_pad

num_of_pairs 100000
num_words:
74020
process_data


In [34]:
for i in range(len(pair_train1)):
    for j in range(que_pad):
        pair_train1[i][j]=word2idx[pair_train1[i][j]]

for i in range(len(pair_train2)):
    for j in range(ans_pad):
        pair_train2[i][j]=word2idx[pair_train2[i][j]]


In [35]:
train_x=[]
train_y=[]
pad_sequence=[word2idx['PAD']]*ans_pad
for i in range(len(pair_train1)):
    for j in range(ans_pad):
        forward=pair_train1[i][:ans_pad]
        backward=pad_sequence[j:ans_pad]
        #print(backward)
        train_x.append(forward+backward+pair_train2[i][:j]) 
        train_y.append([pair_train2[i][j]])

 
train_x=np.array(train_x)
train_y=np.array(train_y)

In [39]:
print(train_x.shape)
print(train_y.shape)
def get_model():
    dim=256
    inputs = Input(shape=(que_pad,))
    g_emb=Embedding(num_words+1,dim,mask_zero=True, input_length=(que_pad))(inputs)
    decoder = GRU(dim)(g_emb)
    decoder = Dense(num_words,activation="softmax")(decoder)
    model = Model(inputs=inputs , outputs=decoder)
    return model

(1000000, 20)
(1000000, 1)


In [40]:
def ppx(y_true, y_pred):
     loss = K.sparse_categorical_crossentropy(y_true, y_pred)
     perplexity = K.cast(K.pow(math.e, K.mean(loss, axis=-1)), K.floatx())
     return perplexity
   

g_model=get_model()
#sampling_model= multi_gpu_model(get_model(), gpus=2)
g_model.compile(loss=ppx, optimizer='adam',metrics=['accuracy'])
earlyStopping=EarlyStopping(monitor='loss', patience=2, verbose=2, mode='auto')

In [41]:
g_model.fit(train_x,train_y, epochs=49, batch_size=512,validation_split=0.2,verbose=1,callbacks=[earlyStopping])

Train on 800000 samples, validate on 200000 samples
Epoch 1/49
800000/800000 [==============================] - 144s 179us/step - loss: 1057.1888 - acc: 0.9978 - val_loss: 398.9773 - val_acc: 0.9991
Epoch 2/49
800000/800000 [==============================] - 141s 176us/step - loss: 303.1695 - acc: 0.5446 - val_loss: 311.7761 - val_acc: 0.3936
Epoch 3/49
800000/800000 [==============================] - 140s 175us/step - loss: 219.7435 - acc: 0.3908 - val_loss: 261.2840 - val_acc: 0.3614
Epoch 4/49
800000/800000 [==============================] - 140s 175us/step - loss: 163.2398 - acc: 0.3499 - val_loss: 268.2411 - val_acc: 0.2751
Epoch 5/49
800000/800000 [==============================] - 141s 176us/step - loss: 120.4709 - acc: 0.3249 - val_loss: 236.0833 - val_acc: 0.2838
Epoch 6/49
800000/800000 [==============================] - 140s 176us/step - loss: 84.8350 - acc: 0.3099 - val_loss: 237.2594 - val_acc: 0.2684
Epoch 7/49
800000/800000 [==============================] - 141s 176us/s

KeyboardInterrupt: 

In [42]:
s_model=g_model #prepare assignment

In [52]:
def output_sequence(pair_train1,pair_train2,num,g_model):
    word2=[]
    test=[pair_train1[num]]
    #print(test)
    test=np.array(test)
    index=g_model.predict(test)
    index=np.argmax(index[0],axis=-1)      
    word2.append(index)
    for i in range(ans_pad-1):
        test=np.delete(test,ans_pad,1)
        test=np.concatenate([test,[[index]]],axis=1)
        index=g_model.predict(test)
        index=np.argmax(index[0],axis=-1)
        word2.append(index)
        if str(index) == str(word2idx['EOS']):
              break
    que=[]
    sample=[]
    test=[pair_train1[num]+pair_train2[num]]
    for g in test[0]:
          for value, age in word2idx.items():
                if age == g:
                	que.append(value)
    for g in word2:
          for value, age in word2idx.items():
                if age == g:
                	sample.append(value)
    print('question')
    print(''.join(que))
    print('RAP_model') 
    print(''.join(sample))
    que=que[0:20]+['   ans:   ']+que[ans_pad:]      
    return  ''.join(que),''.join(sample)
update_g=len(pair_train1)
for i in range(5):
    output_sequence(pair_train1,pair_train2,random.randint(0,random.randint(0,pair-1)),g_model)

old_result=0

question
学校发的蚊帐终于派上用场了PADPADPADPADPADPADPADPADPADPADPADPAD我们高中的时候就用上了PADPADPAD
RAP_model
我也有一个，有木有。
question
分享图片老板，再来一桶PADPADPADPADPADPADPADPADPADPADPADPADPAD哈哈哈。蒙到无语了。喜悦啊。PAD
RAP_model
一个字——不要这么搞。PADPAD
question
不要相信谣言，因为真相更惊人。PADPADPADPADPADPADPADPADPADPADPAD谣言止于智者，智者却要止于真相PAD
RAP_model
精辟，精辟，方舟子。PADPADPADPAD
question
他妈的，这战术，坑爹啊。PADPADPADPADPADPADPADPADPADPADPAD唯一需要用到的技术就是淡定……PAD
RAP_model
这是什么意思的名字PADPADPADPAD
question
【娱乐一下】男生的价格计算方法PADPADPADPADPADPADPADPADPADPADPAD原来只是正常而已？！哈哈哈。PADPAD
RAP_model
关键是男性错误主义价值。PADPADPAD


In [44]:
dim=128
d_input1=Input(shape=(que_pad,))
d_input2=Input(shape=(1,))
#,mask_zero=True
con=concatenate([d_input1,d_input2],axis=1)
d_emb=Embedding(num_words+1,dim, input_length=(que_pad+1))(con)
sent_representation=LSTM(dim)(d_emb)
#sent_representation=GlobalMaxPooling1D()(cnn)
probabilities = Dense(2, activation='softmax')(sent_representation)
discriminator = Model([d_input1,d_input2] , probabilities)
discriminator.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])

In [45]:
def distribution_index(fake):
    fake_idx=[]
    for i in fake:
    	w=np.argmax(i)
    	fake_idx.append([w])
    fake_idx=np.array(fake_idx)
    return fake_idx


In [46]:
def rollout_reward(train_x,train_y,count,candidate):
    rollout_sample=[]
    new_input=np.array(train_x)
    #print(new_input.shape)
    new_x=[]
    new_y=[]
    new_input_x=[]
    new_input_y=[]
    reward=[]
    
    for i in range(count):
        fake=g_model.predict(new_input)
        old_input=new_input
        new_input=[]
        for k in range(len(fake)):
            #print(k)
            index=np.argsort(fake[k])
            index=index[::-1]#排序由機率大到小
            for key,j in enumerate(index[0:candidate]):
                if i==0:
                    new_input_x.append(train_x[0])
                    new_input_y.append([j])
                new_x.append(new_input)
                new_y.append([j])
                con=np.delete(old_input[k],ans_pad,0)
                con=np.concatenate([con,[j]],axis=0)
                new_input.append(con)
                
        new_input=np.array(new_input)
        
    new_y=np.array(new_y)
    #print(new_input,new_y) 
    k=len(new_input)//candidate
    for i in range(candidate):
        vector=discriminator.predict([new_input[k*i:k*(i+1)],
                                  new_y[k*i:k*(i+1)]])[:,0]
        reward.append(np.mean(vector))
#     reward.append(discriminator.predict([train_x,train_y])[0][0])
    #print(reward)
    reward=reward-np.mean(reward)
#     new_input_x.append(train_x[0])
#     new_input_y.append(train_y[0])
    return new_input_x,new_input_y,reward.tolist()

In [47]:
def greedy_search(g_model,train_x,train_y):
    new_trainX=[]
    new_trainY=[]
    for key,i in enumerate(tqdm_notebook(train_x)):
        if key%ans_pad==0:
            train_w=i
        else:
            #train_w[ans_pad+key%ans_pad-1]=index
            train_w=np.delete(train_w,ans_pad,0)   
            train_w=np.concatenate([train_w,[index]],axis=0)    
        index=g_model.predict(np.array([train_w]))
        index=np.argmax(index[0],axis=-1)
        new_trainX.append(train_w)    
        new_trainY.append([index])
    new_trainX=np.array(new_trainX)
    new_trainY=np.array(new_trainY)
    return new_trainX,new_trainY
def train_d(discriminator,train_x,train_y,X,Y):
    earlyStopping=EarlyStopping(monitor='loss', patience=1, verbose=2, mode='auto')
    n = len(train_x)
    YT = np.zeros([n*2,2])
    YT[0:n,1] = 1
    YT[n:,0] = 1
    #fake=distribution_index(fake)
    XT=np.concatenate([X,train_x])
    XT2=np.concatenate([Y,train_y])
    result=discriminator.fit([XT,XT2],YT, epochs=1,shuffle=True, batch_size=32, verbose=0,callbacks=[earlyStopping])
    return result.history['acc'][-1]

In [48]:
def return_all_mean(train_x,train_y):
    fake=g_model.predict(train_x)
    #per=s_model.evaluate(train_x,train_y,batch_size=512)

    #per=1/per[0]
    mean=discriminator.predict([train_x,distribution_index(fake)])
    return np.mean(mean[:,1])

In [50]:
epoch=0
#for s in range(120):
for x in range(10):
#     print("greedy-search")
#     X,Y=greedy_search(g_model,train_x[::ans_pad],train_y[::ans_pad])
#     print('d-step')
#     result=train_d(discriminator,train_x[::ans_pad],train_y[::ans_pad],X,Y)
#     del X,Y
#     print(result)

#     print('finishsearch')
    count=0   
    dis_x=[]
    dis_y=[]
    reward=[]
    for g in tqdm_notebook(range(int(len(train_x)))):
        if g%ans_pad==0:
             code=0
        new_trainX=[]
        new_trainY=[]
        if g%ans_pad==0:
            train_w=train_x[g]    
        index=g_model.predict(np.array([train_w]))
        index=np.argmax(index[0],axis=-1)
        new_trainX.append(train_w)    
        new_trainY.append([index])
        if g%ans_pad!=0:
            train_w=np.delete(train_w,ans_pad,0)   
            train_w=np.concatenate([train_w,[index]],axis=0)
        s=rollout_reward(new_trainX,new_trainY,(ans_pad-g%ans_pad),2)    
        dis_x.extend(s[0])
        dis_y.extend(s[1]) 
        reward.extend(s[2])
        code+=1 
        if g%10000==0 and g!=0:
            print('batch',g)
            g_model.fit(np.array(dis_x),np.array(dis_y),epochs=1,batch_size=256,sample_weight=np.array(reward),verbose=0)
            dis_x=[]
            dis_y=[]
            reward=[]
            for i in range(3):
                output_sequence(pair_train1,pair_train2,random.randint(0,random.randint(0,pair-1)),g_model)
    epoch=epoch+1
    print('epoch',epoch) 
    #print("--- %s seconds ---" % (time.time() - start_time))   

KeyboardInterrupt: 

In [53]:
for i in range(3):
    output_sequence(pair_train1,pair_train2,random.randint(0,random.randint(0,pair-1)),g_model)

question
有移动互联网记者和新媒体编辑跳槽么PADPADPADPADPADPADPADPADPADPADPAD可考虑啊在上海吗待遇怎样PADPAD
RAP_model
的感觉好有爱啊，好有趣PAD
question
40年前的数学课本，很给力！PADPADPADPADPADPADPADPADPADPADPAD仔细一看，确实给力PADPADPADPAD
RAP_model
地主阶级是巧合的！PADPADPADPAD
question
据说Linsanity有官方翻译了：林彪！PADPADPADPADPADPADPADPADPADPADPAD那他未来儿子起名…Linfruit：林立果PAD
RAP_model
东北人在中国运动员不要说话PADPADPADPAD
